In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData

from core import services
from core import utils
from core import controller

import os
from groq import Groq
from groq import AsyncGroq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
async_client=AsyncGroq(api_key=os.getenv("GROQ_API_KEY"))

async def async_transcribe(file_path):
    # load the file as a binary
    audio_file=open(file_path, "rb")
    # create the transcription
    transcription = await async_client.audio.transcriptions.create(
      file=(file_path, audio_file.read()),
      model="whisper-large-v3",
      response_format="verbose_json",
    )

    # return the transcription
    return transcription

In [13]:
# SPLIT UP THE AUDIO FILE AND SAVE IT
mp4_test_folder='destinyapp/working_folder/mp4_2_mp3'
files=os.listdir(mp4_test_folder)
for file in files:

    ffmpeg_command = f"ffmpeg -i {mp4_test_folder}/{file} -vn -acodec libmp3lame -b:a 192k {mp4_test_folder}/{os.path.splitext(file)[0]}.mp3"
    os.system(ffmpeg_command)
    break


In [14]:
# SPLIT UP THE AUDIO FILE AND SAVE IT
test_folder='destinyapp/working_folder/testing'
files=os.listdir(test_folder)
for file in files:
    os.remove(test_folder+'/'+file)

file_path="D:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject\destinyapp\working_folder\working_audio\merged_audio.mp3"
ffmpeg_command="ffmpeg -i "+file_path+" -f segment -segment_time 1200 -c copy "+test_folder+"/output_%03d.mp3"
os.system(ffmpeg_command)

0

In [15]:
# Produce the transcripts
files=os.listdir(test_folder)
tasks=[]
for file in files:
    tasks.append(async_transcribe(test_folder+'/'+file))
task_results=await asyncio.gather(*tasks)

In [16]:
# make full transcript
full_regular_transcript=""
for result in task_results:
    full_regular_transcript+=result.text

In [17]:
print(full_regular_transcript)

 I don't know if you saw, but I just watched Asmongold's video about Assange saying, Cracker isn't a racial slur. Isn't racism racism? Kind of, yeah. Racism is experienced in different ways by different people, and different power structures can create different types of racism. Like, as a white person in the United States, you'll never experience the same type of structural or systemic racism as a black person will. But on an individual level, you can experience racism of all sorts of varying degrees that could be worse than some of the structural or systemic stuff. structural and systemic racism is a very weak form of racism like it's pervasive and it's everywhere on a large scale and it can impact everything but the impact on an individual could be quite weak like um i would say that like uh individual racism is like the strong nuclear force and um and systemic racism is like gravity um which i think is the weakest of the four do we even use fundamental forces anymore or has that al

In [9]:
# Make the linked transcript
linked_transcript=""
video_id="test"
transcript=""
new_lines=0
word_count=0
time=0
# split the transcript every 20 words
words_full_transcript=full_regular_transcript.split(" ")
split_full_transcript=[" ".join(words_full_transcript[i:i+20]) for i in range(0, len(words_full_transcript), 20)]
for utterance in split_full_transcript:
    time+=1
    text=utterance
    val=100
    base_link="https://www.youtube.com/watch?v={video_id}&t={time}s"
    link=f"https://youtu.be/{video_id}?t={time}"
    linked_transcript+="<a href=\""+link+"\" target=\"_blank\" class=\"underline-hover\">"+text+"</a>"

In [10]:
# save json to testin
test_folder='destinyapp/working_folder/test_transcript/transcript.json'

import json
with open(test_folder, 'w') as f:
    json.dump({"transcript":full_regular_transcript, "linked_transcript":linked_transcript}, f)
